# Importing required libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
# Define the paths to your dataset directories
real_dir = 'dataset/train/training_real'
fake_dir = 'dataset/train/training_fake'

# Define image size and batch size
IMG_SIZE = (600, 600)
BATCH_SIZE = 32

# Load and preprocess images
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMG_SIZE)
    image /= 255.0  # Normalize to [0,1] range
    return image

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = preprocess_image(image)
    return image, label

# Create labeled datasets
real_ds = tf.data.Dataset.list_files(real_dir + '/*')
fake_ds = tf.data.Dataset.list_files(fake_dir + '/*')

real_ds = real_ds.map(lambda x: load_and_preprocess_image(x, 1))
fake_ds = fake_ds.map(lambda x: load_and_preprocess_image(x, 0))

# Combine and shuffle the datasets
full_ds = real_ds.concatenate(fake_ds)
full_ds = full_ds.shuffle(buffer_size=100)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(600, 600, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Calculate the number of elements in the dataset
num_samples = sum(1 for _ in full_ds)

# Calculate the number of validation samples
validation_split = 0.5
num_val_samples = int(validation_split * num_samples)
num_train_samples = num_samples - num_val_samples

# Create the training and validation datasets
val_ds = full_ds.take(num_val_samples).batch(BATCH_SIZE)
train_ds = full_ds.skip(num_val_samples).batch(BATCH_SIZE)

# Prefetch to improve performance
train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.experimental.AUTOTUNE)

# Train the model
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10


2024-05-30 18:52:38.779569: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


27/27 ━━━━━━━━━━━━━━━━━━━━ 1137s 43s/step - accuracy: 0.7921 - loss: 10.7456 - val_accuracy: 0.0000e+00 - val_loss: 19.4706
Epoch 2/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 1343s 50s/step - accuracy: 0.5969 - loss: 3.3136 - val_accuracy: 0.0000e+00 - val_loss: 8.4386
Epoch 3/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 1795s 68s/step - accuracy: 0.5792 - loss: 1.7665 - val_accuracy: 0.0000e+00 - val_loss: 94.3482
Epoch 4/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 918s 34s/step - accuracy: 0.6142 - loss: 16.4565 - val_accuracy: 0.0000e+00 - val_loss: 40.6983
Epoch 5/10


In [88]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 598, 598, 32)      896       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 299, 299, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 297, 297, 64)      18496     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 148, 148, 64)      0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 146, 146, 128)     73856     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 73, 73, 128)      

In [89]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = preprocess_image(image)
    return image

test_dir = '/content/drive/MyDrive/M.Tech Data Science and Machine Learning/Semester 2/DL/Hackathon/dataset/test'
test_ds1 = tf.data.Dataset.list_files(test_dir + '/*')
test_ds2 = test_ds1.map(load_and_preprocess_image)

# Batch and prefetch the test dataset
test_ds = test_ds2.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [90]:
# Make predictions on the test dataset
predictions = model.predict(test_ds)

11/11 [==============================] - 7s 624ms/step


In [91]:
# Convert predictions to binary labels (0 or 1)
binary_predictions = (predictions > 0.5).astype(int)

# Display the predicted labels
print(binary_predictions.reshape(1,-1)[0])

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [92]:
import numpy as np
# Get file names from file paths and convert tensors to strings
test_file_names = [file_path.numpy().decode('utf-8').split('/')[-1] for file_path in test_ds1]

# Now, test_file_names contains the file names of the test images as strings


In [93]:
print(test_file_names)

['313.jpg', '91.jpg', '94.jpg', '37.jpg', '253.jpg', '183.jpg', '132.jpg', '188.jpg', '93.jpg', '160.jpg', '4.jpg', '252.jpg', '271.jpg', '99.jpg', '33.jpg', '113.jpg', '112.jpg', '3.jpg', '231.jpg', '296.jpg', '118.jpg', '280.jpg', '119.jpg', '144.jpg', '236.jpg', '213.jpg', '310.jpg', '276.jpg', '28.jpg', '54.jpg', '131.jpg', '309.jpg', '270.jpg', '288.jpg', '318.jpg', '285.jpg', '329.jpg', '198.jpg', '302.jpg', '63.jpg', '104.jpg', '107.jpg', '111.jpg', '186.jpg', '255.jpg', '25.jpg', '85.jpg', '19.jpg', '79.jpg', '166.jpg', '26.jpg', '129.jpg', '240.jpg', '242.jpg', '154.jpg', '116.jpg', '194.jpg', '97.jpg', '39.jpg', '212.jpg', '136.jpg', '282.jpg', '47.jpg', '56.jpg', '226.jpg', '139.jpg', '50.jpg', '323.jpg', '263.jpg', '281.jpg', '227.jpg', '126.jpg', '146.jpg', '135.jpg', '14.jpg', '224.jpg', '262.jpg', '0.jpg', '58.jpg', '42.jpg', '211.jpg', '209.jpg', '294.jpg', '327.jpg', '275.jpg', '232.jpg', '80.jpg', '290.jpg', '69.jpg', '320.jpg', '241.jpg', '7.jpg', '34.jpg', '196.jpg'

In [94]:
# Combine file paths with predictions
predictions_with_filenames = list(zip(test_file_names, binary_predictions.reshape(1,-1)[0]))

In [95]:
predictions_with_filenames

[('313.jpg', 1),
 ('91.jpg', 1),
 ('94.jpg', 1),
 ('37.jpg', 1),
 ('253.jpg', 1),
 ('183.jpg', 1),
 ('132.jpg', 1),
 ('188.jpg', 1),
 ('93.jpg', 1),
 ('160.jpg', 1),
 ('4.jpg', 1),
 ('252.jpg', 1),
 ('271.jpg', 1),
 ('99.jpg', 1),
 ('33.jpg', 1),
 ('113.jpg', 1),
 ('112.jpg', 1),
 ('3.jpg', 1),
 ('231.jpg', 1),
 ('296.jpg', 1),
 ('118.jpg', 1),
 ('280.jpg', 1),
 ('119.jpg', 1),
 ('144.jpg', 1),
 ('236.jpg', 1),
 ('213.jpg', 1),
 ('310.jpg', 1),
 ('276.jpg', 1),
 ('28.jpg', 1),
 ('54.jpg', 1),
 ('131.jpg', 1),
 ('309.jpg', 1),
 ('270.jpg', 1),
 ('288.jpg', 1),
 ('318.jpg', 1),
 ('285.jpg', 1),
 ('329.jpg', 1),
 ('198.jpg', 1),
 ('302.jpg', 1),
 ('63.jpg', 1),
 ('104.jpg', 1),
 ('107.jpg', 1),
 ('111.jpg', 1),
 ('186.jpg', 1),
 ('255.jpg', 1),
 ('25.jpg', 1),
 ('85.jpg', 1),
 ('19.jpg', 1),
 ('79.jpg', 1),
 ('166.jpg', 1),
 ('26.jpg', 1),
 ('129.jpg', 1),
 ('240.jpg', 1),
 ('242.jpg', 1),
 ('154.jpg', 1),
 ('116.jpg', 1),
 ('194.jpg', 1),
 ('97.jpg', 1),
 ('39.jpg', 1),
 ('212.jpg', 1),


In [96]:
import pandas as pd

submission  = {'file_id':[],'label':[]}

for file_id,label in predictions_with_filenames:
  submission['file_id'].append(int(file_id.split('.')[0]))
  submission['label'].append(label)

submission_pd = pd.DataFrame(submission)
submission_pd.head()

,file_id,label
0,313,1
1,91,1
2,94,1
3,37,1
4,253,1


In [97]:
submission_pd.to_csv('Submission.csv',index=False)